# Memo Conversion
* model：GLM-4
* psychological counseling report to counseling note 

In [1]:
import threading
import json
import os
from tqdm import tqdm
import concurrent.futures
import requests
from dotenv import load_dotenv
from zhipuai import ZhipuAI

In [ ]:
system_message = '''
# Role 
你是一位有着二十年从业经验的心理咨询师，擅长重建心理咨询场景。
## Attention
您负责基于心理咨询报告，设计咨询笔记。

## Skills
### Skill 1：解析心理咨询报告
- 详细读取和解析来访者心理咨询报告的所有内容。
- 理解和把握来访者的基本情况、心理问题等关键信息。

### Skill 2：设计咨询笔记
- 基于咨询报告的消化和理解，设计出专业的咨询笔记。
- 咨询笔记包括："咨询笔记："
   -  来访者的基本情况： "一、来访者的基本情况"
   -  来访者的心理问题： "二、来访者的心理问题"
   -  咨询方案： "三、咨询方案"
       - 咨询目标
       - 心理咨询技术
       - 具体执行计划
   -  经验感想与反思： "四、经验感想与反思"

## Constrains
- 尽可能使用专业的心理咨询词汇和术语
- 遵循心理咨询的隐私准则
- <咨询方案>中来访者的基本情况务必为来访者基本情况的总结
- <咨询方案>中来访者的心理问题务必为来访者具体的心理问题

请深呼吸并逐步分析心理咨询报告，基于其内容设计咨询笔记。
'''

In [ ]:
interrupted = threading.Event()
pbar = None

In [ ]:
def get_completion_from_messages(case, i, system_message):

    try:
        if interrupted.is_set():
            raise KeyboardInterrupt
        
        case_str = json.dumps(case, ensure_ascii=False)
        
        if not case_str:
            print(f"Error at {i}: invalid")
            with open("error_log.txt", "a") as f:
                f.write(f"\nError at case {i}: invalid")
            return
        
        case_str = system_message + '\n' + '心理咨询报告:"""' + case_str + '"""'

        messages = [{"role": "user", "content": case_str}]
        
        load_dotenv()
        client = ZhipuAI(api_key=os.environ["ZhipuAI_API_KEY"])

        response = client.chat.completions.create(
            model="glm-4",
            messages = messages,
            temperature= 0.85,
            top_p= 0.7,
            incremental=True
        )
        
        with open(f"response_case_{i}.txt", 'w') as f:
            ans = response['data']['choices'][0]['content']
            f.writelines(ans)

    except KeyboardInterrupt:
        print(f"Error at case {i}: Interrupted")
        with open("error_log.txt", "a") as f:
            f.write(f"\nError at case {i}: Interrupted")
        return
    
    except Exception as e:
        print(f"Error at case {i}: {e}")
        with open("error_log.txt", "a") as f:
            f.write(f"\nError at case {i}: {e}")

    pbar.update()

def load_case(filepath):
    with open(filepath, 'r') as f:
        case = json.load(f)
    return case


In [ ]:
if __name__ == "__main__":
    path = ''   # report file path
    threads = []
    pbar = tqdm(total=3133, desc="Processing", dynamic_ncols=True, unit=" case", lock_args=(False,))

    try:
        with concurrent.futures.ThreadPoolExecutor() as executor:
            for i in range(0, 3133): 
                case_file_path = os.path.join(path, f'case_{i}.json')
                case = load_case(case_file_path)
                executor.submit(get_completion_from_messages, case, i, system_message)

    except KeyboardInterrupt:
        print("Interrupted by user")
        interrupted.set()

    pbar.close()